# Bottle Processing
Author: Andrew Reed

### Motivation:
Independent verification of the suite of physical and chemical observations provided by OOI are critical for the observations to be of use for scientifically valid investigations. Consequently, CTD casts and Niskin water samples are made during deployment and recovery of OOI platforms, vehicles, and instrumentation. The water samples are subsequently analyzed by independent labs for  comparison with the OOI telemetered and recovered data.

However, currently the water sample data routinely collected and analyzed as part of the OOI program are not available in a standardized format which maps the different chemical analyses to the physical measurements taken at bottle closure. Our aim is to make these physical and chemical analyses of collected water samples available to the end-user in a standardized format for easy comprehension and use, while maintaining the source data files. 

### Approach:
Generating a summary of the water sample analyses involves preprocessing and concatenating multiple data sources, and accurately matching samples with each other. To do this, I first preprocess the ctd casts to generate bottle (.btl) files using the SeaBird vendor software following the SOP available on Alfresco. 

Next, the bottle files are parsed using python code and the data renamed following SeaBird's naming guide. This creates a series of individual cast summary (.sum) files. These files are then loaded into pandas dataframes, appended to each other, and exported as a csv file containing all of the bottle data in a single data file.

### Data Sources/Software:

* **sbe_name_map**: This is a spreadsheet which maps the short names generated by the SeaBird SBE DataProcessing Software to the associated full names. The name mapping originates from SeaBird's SBE DataProcessing support documentation.

* **Alfresco**: The Alfresco CMS for OOI at alfresco.oceanobservatories.org is the source of the ctd hex, xmlcon, and psa files necessary for generating the bottle files needed to create the sample summary sheet.

* **SBEDataProcessing-Win32**: SeaBird vendor software for processing the raw ctd files and generating the .btl files.


In [1]:
# Import packages used in this notebook
import os, sys, re
import pandas as pd
import numpy as np

In [2]:
# Load the name mapping for the column names
# Specifiy the local directory
sbe_name_map = pd.read_excel('/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')
# sbe_name_map = pd.read_excel('C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')

In [8]:
basepath = '/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/'
array = 'Pioneer/'
cruise = 'Pioneer-05/Leg_1/'
water_dir = 'Water Sampling/'
ctd_dir = 'ctd/'

In [9]:
# Specify the local directory where the bottle (.btl) files are stored for a particular cruise
# dirpath = 'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/ctd/'
btlpath = basepath+array+cruise+ctd_dir
summary_sheet_path = basepath+array+cruise+water_dir+'Pioneer2_KN-217_sampling_log-1.xlsx'
salts_and_o2_path = basepath+array+cruise+water_dir+'Salts and O2/'
nutrients_path = basepath+array+cruise+water_dir+'Pioneer II Nutrient Data 2014.xlsx'
chl_path = basepath+array+cruise+water_dir+'CHLs KN-214.xls'

In [10]:
os.listdir(btlpath)

['at31005.hdr',
 'at990.bl',
 '09P-0749_at31abc.xmlcon',
 'at31001.bl',
 'at31001.btl',
 'at31001.hdr',
 'at31001.hex',
 'at31001.ros',
 'at31001.sum',
 'AT31001.XMLCON',
 'at31002.bl',
 'at31002.btl',
 'at31002.hdr',
 'at31002.hex',
 'at31002.ros',
 'at31002.sum',
 'AT31002.XMLCON',
 'at31003.bl',
 'at31003.btl',
 'at31003.hdr',
 'at31003.hex',
 'at31003.ros',
 'at31003.sum',
 'AT31003.XMLCON',
 'at31004.bl',
 'at31004.btl',
 'at31004.hdr',
 'at31004.hex',
 'at31004.ros',
 'at31004.sum',
 'AT31004.XMLCON',
 'at31005.bl',
 'at31005.btl',
 'at31005.hex',
 'at31005.ros',
 'at31005.sum',
 'AT31005.XMLCON',
 'at31006.bl',
 'at31006.btl',
 'at31006.hdr',
 'at31006.hex',
 'at31006.ros',
 'at31006.sum',
 'AT31006.XMLCON',
 'at31007.bl',
 'at31007.btl',
 'at31007.hdr',
 'at31007.hex',
 'at31007.ros',
 'at31007.sum',
 'AT31007.XMLCON',
 'at31008.bl',
 'at31008.btl',
 'at31008.hdr',
 'at31008.hex',
 'at31008.ros',
 'at31008.sum',
 'AT31008.XMLCON',
 'at31009.bl',
 'at31009.hdr',
 'at31009.hex',


In [6]:
# Parse the data for the start_time
def parse_header(header):
    """
    Parse the header of bottle (.btl) files to get critical information
    for the summary spreadsheet.
    
    Args:
        header - an object containing the header of the bottle file as a list of
            strings, split at the newline.
    Returns:
        hdr - a dictionary object containing the start_time, filename, latitude,
            longitude, and cruise id.
    """
    hdr = {}
    for line in header:
        if 'start_time' in line.lower():
            start_time = pd.to_datetime(re.split('= |\[',line)[1])
            hdr.update({'Start Time':start_time.strftime('%Y-%m-%dT%H:%M:%SZ')})
        elif 'filename' in line.lower():
            hex_name = re.split('=',line)[1].strip()
            hdr.update({'Hex name':hex_name})
        elif 'latitude' in line.lower():
            start_lat = re.split('=',line)[1].strip()
            hdr.update({'Start Latitude':start_lat})
        elif 'longitude' in line.lower():
            start_lon = re.split('=',line)[1].strip()
            hdr.update({'Start Longitude':start_lon})
        elif 'cruise id' in line.lower():
            cruise_id = re.split(':',line)[1].strip()
            hdr.update({'Cruise ID':cruise_id})
        else:
            pass
    
    return hdr
        

In [11]:
# Now write a function to autopopulate the bottle summary sample sheet
files = [x for x in os.listdir(btlpath) if '.btl' in x]
for filename in files:
    filepath = os.path.abspath(btlpath+filename)
    
    # Load the raw content into memory
    with open(filepath) as file:
        content = file.readlines()
    content = [x.strip() for x in content]
    
    # Now parse the file content
    header = []
    columns = []
    data = []
    for line in content:
        if line.startswith('*') or line.startswith('#'):
            header.append(line)
        else:
            try:
                float(line[0])
                data.append(line)
            except:
                columns.append(line)
    
    # Parse the header
    hdr = parse_header(header)
    
    # Parse the column identifiers
    column_dict = {}
    for line in columns:
        for i,x in enumerate(line.split()):
            try:
                column_dict[i] = column_dict[i] + ' ' + x
            except:
                column_dict.update({i:x})
    
    # Parse the bottle data based on the column header locations
    data_dict = {x:[] for x in column_dict.keys()}

    for line in data:
        if line.endswith('(avg)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            for i,x in enumerate(values):
                data_dict[i].append(x)
        elif line.endswith('(sdev)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            data_dict[1].append(values[0])
        else:
            pass
            
    data_dict[1] = [' '.join(item) for item in zip(data_dict[1][::2],data_dict[1][1::2])]
    
    # With the parsed data and column names, match up the data and column
    # based on the location
    results = {}
    for key,item in column_dict.items():
        values = data_dict[key]
        results.update({item:values})
        
    # Put the results into a dataframe
    df = pd.DataFrame.from_dict(results)
        
    # Now add the parsed info from the header files into the dataframe
    for key,item in hdr.items():
        df[key] = item
        
    # Get the cast number
    cast = filename[filename.index('.')-3:filename.index('.')]
    df['Cast #'] = str(cast).zfill(3)
    
    # Generate a filename for the summary file
    outname = filename.split('.')[0] + '.sum'
    
    # Save the results
    df.to_csv(btlpath+outname)

In [14]:
hdr

{'Hex name': 'C:\\data\\ctd\\at31008.hex',
 'Start Latitude': '40 21.90 N',
 'Start Longitude': '070 46.50 W',
 'Start Time': '2015-10-16T14:35:11Z'}

In [12]:
# Now, for each "summary" file, load and append to each other
df = pd.DataFrame()
for file in os.listdir(btlpath):
    if '.sum' in file:
        df = df.append(pd.read_csv(btlpath+file))
    else:
        pass

In [13]:
df.head()

,Unnamed: 0,Bottle Position,Date Time,PrDM,DepSM,Latitude,Longitude,T090C,T190C,C0S/m,...,Sal11,Sbeox0ML/L,OxsolML/L,CStarAt0,CStarTr0,Hex name,Start Latitude,Start Longitude,Start Time,Cast #
0,0,1,Oct 12 2015 16:39:05,443.326,439.460,39.94014,-70.8834,6.0237,6.0239,3.463375,...,35.0512,4.4897,6.90263,0.2333,94.3332 (avg),C:\data\ctd\at31001.hex,39 56.41 N,070 53.00 W,2015-10-12T15:23:02Z,1
1,1,2,Oct 12 2015 16:39:23,443.278,439.413,39.94014,-70.8834,6.0226,6.0234,3.463267,...,35.0512,4.4967,6.90281,0.2632,93.6523 (avg),C:\data\ctd\at31001.hex,39 56.41 N,070 53.00 W,2015-10-12T15:23:02Z,1
2,2,3,Oct 12 2015 16:45:09,251.878,249.798,39.94014,-70.8834,9.3260,9.3490,3.774700,...,35.1904,2.9037,6.40054,0.1663,95.9263 (avg),C:\data\ctd\at31001.hex,39 56.41 N,070 53.00 W,2015-10-12T15:23:02Z,1
3,3,4,Oct 12 2015 16:45:17,252.137,250.054,39.94014,-70.8834,9.3717,9.3705,3.779986,...,35.1959,2.9237,6.39372,0.1657,95.9419 (avg),C:\data\ctd\at31001.hex,39 56.41 N,070 53.00 W,2015-10-12T15:23:02Z,1
4,4,5,Oct 12 2015 16:55:13,52.160,51.755,39.94014,-70.8834,21.8404,21.8252,5.073489,...,35.7202,4.7131,4.98466,0.2745,93.3688 (avg),C:\data\ctd\at31001.hex,39 56.41 N,070 53.00 W,2015-10-12T15:23:02Z,1


In [ ]:
sbe_name_map['Short Name'].apply(lambda x: str(x).lower());

In [ ]:
# Rename the column title using the sbe_name_mapping 
for colname in list(df.columns.values):
    try:
        fullname = list(sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == colname.lower()) == True]['Full Name'])[0]
        df.rename({colname:fullname},axis='columns',inplace=True)
    except:
        pass

In [ ]:
df.rename(columns={'Bottle Position':'Niskin #'},inplace=True)
df['Niskin #'] = df['Niskin #'].apply(lambda x: str( int(x) ) )
df.drop(columns='Unnamed: 0',inplace=True)
df['Cast #'] = df['Cast #'].apply(lambda x: str(x).zfill(3) )

In [ ]:
df

In [ ]:
df.to_csv(btlpath+'CTD_Summary.csv')

### Oxygen & Salinity 
Now, we need to add the 

In [ ]:
def clean_sal_files(dirpath):
    
    # Run check if files are held in excel format or csvs
    csv_flag = any(files.endswith('.SAL') for files in os.listdir(dirpath))
    if csv_flag:
        for files in os.listdir(dirpath):
            sample = []
            salinity = []
            with open(basepath+array+cruise+'Water Sampling/Salts and O2/'+leg+files) as file:
                data = file.readlines()
                for ind1,line in enumerate(data):
                    if ind1 == 0:
                        strs = data[0].replace('"','').split(',')
                        cruisename = strs[0]
                        station = strs[1]
                        cast = strs[2]
                        case = strs[8]
                    elif int(line.split()[0]) == 0:
                        pass
                    else:
                        strs = line.split()
                        sample.append(strs[0])
                        salinity.append(strs[2]) 
                # Generate a pandas dataframe to populate data
                data_dict = {'Cruise ID':cruisename,'Station #':station,'Cast #':cast,'Case':case,'Sample ID':sample,'Salinity [psu]':salinity}
                df = pd.DataFrame.from_dict(data_dict)
                df.to_csv(file.name.replace('.','')+'.csv')
    
    else:
        # If the files are already in excel spreadsheets, they've been cleaned into a
        # logical tabular format
        pass
    

def process_sal_files(dirpath):
    
    # Check if the files are excel files or not
    excel_flag = any(files.endswith('SAL.xlsx') for files in os.listdir(dirpath))
    # Initialize a dataframe for processing the salinity files
    df = pd.DataFrame()
    if excel_flag:
        for file in os.listdir(dirpath):
            if 'SAL.xlsx' in file:
                df = df.append(pd.read_csv(dirpath+file))
        df.rename({'Cruise':'Cruise ID','Station':'Station #','Sample':'Sample ID','Salinity':'Salinity [psu]'},
          axis='columns',inplace=True)
        df.dropna(inplace=True)
        df['Station #'] = df['Station #'].apply(lambda x: str( int(x)).zfill(3))
        df['Niskin #'] = df['Niskin #'].apply(lambda x: str( int(x)))
        df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
    else:
        for file in os.listdir(dirpath):
            if 'SAL.csv' in file:
                df = df.append(pd.read_csv(dirpath+file))
        df.dropna(inplace=True)
        df['Station #'] = df['Station #'].apply(lambda x: str( int(x)).zfill(3))
        df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
        df.drop(columns=[x for x in list(df.columns.values) if 'unnamed' in x.lower()],inplace=True)

    # Save the processed summary file for salinity
    df.to_csv(dirpath+'SAL_Summary.csv')
    
    
def process_oxy_files(dirpath):
    df = pd.DataFrame()
    for file in os.listdir(dirpath):
        if 'OXY' in file:
            df = df.append(pd.read_excel(dirpath+file))
    # Rename and clean up the oxygen data to be uniform across data sets
    df.rename({'Cruise':'Cruise ID','Station':'Station #','Sample#':'Sample ID','Oxy':'Oxygen [mL/L]','Unit':'Units'},
          axis='columns',inplace=True)
    df.dropna(inplace=True)
    df['Station #'] = df['Station #'].apply(lambda x: str( int(x)).zfill(3))
    df['Niskin #'] = df['Niskin #'].apply(lambda x: str( int(x)))
    df['Sample ID'] = df['Sample ID'].apply(lambda x: str( int(x)))
    df['Cruise ID'] = df['Cruise ID'].apply(lambda x: x.replace('O','0'))
    
    # Save the processed summary file for oxygen
    df.to_csv(dirpath+'OXY_Summary.csv')

### CTD Sampling Log

In [ ]:
os.listdir(basepath+array+cruise+water_dir)

In [ ]:
summary_sheet_path = basepath+array+cruise+water_dir+'PIONEER-5_AT-31B_CTD_sampling_logs.xlsx'

In [ ]:
sample_log = pd.read_excel(summary_sheet_path,sheet_name='Summary',header=0)

In [ ]:
sample_log.head()

In [ ]:
# # Rename the column headers
# sample_log.rename(columns=lambda x: 'Sample Log: ' + x.strip(), inplace=True)
# sample_log['Sample Log: Niskin #'] = sample_log['Sample Log: Niskin #'].apply(lambda x: int(x.replace('*','')) if type(x) == str else x )
# sample_log['Sample Log: Niskin #'] = sample_log['Sample Log: Niskin #'].apply(lambda x: x if np.isnan(x) else str( int(x) ) )
# sample_log['Sample Log: Cast #'] = sample_log['Sample Log: Station-Cast #'].apply(lambda x: str(x).zfill(3))

### Nutrient & Chlorophyll Data

In [ ]:
os.listdir(basepath+array+cruise+water_dir)

In [ ]:
chlfile = 'Pioneer-5_AT-31B_chlorophyll_analysis.xlsx'
nutfile = 'Pioneer-5_AT-31B_Nutrient_Lab_Results.xlsx'

In [ ]:
nutrients = pd.read_excel(basepath+array+cruise+water_dir+nutfile)

In [ ]:
nutrients.head()

In [ ]:
chl = pd.read_excel(basepath+array+cruise+water_dir+chlfile)
chl.head()

In [ ]:
os.listdir(basepath+array+cruise+water_dir+'Salts and O2')

In [ ]:
# Load the Salinity and oxygen summaries
sal = pd.read_csv(basepath+array+cruise+water_dir+'Salts and O2/AT31-A/SAL_Summary.csv')
if 'case' in [x.lower() for x in sal.columns.values]:
    sal['Sample ID'] = sal['Case'] + sal['Sample ID'].apply(lambda x: str(x)) 
oxy = pd.read_csv(basepath+array+cruise+water_dir+'Salts and O2/AT31-A/OXY_Summary.csv')
if 'case' in [x.lower() for x in oxy.columns.values]:
    oxy['Sample ID'] = oxy['Case'] + oxy['Sample ID'].apply(lambda x: str(x)) 

In [ ]:
sal.head()

In [ ]:
sample_log.head()

In [ ]:
# Now need to mak
sample_log = sample_log.merge(sal[['Station #','Sample ID','Salinity [psu]']], how='left', left_on=['Station-Cast #','Salts Bottle #'], right_on=['Station #','Sample ID'])

In [ ]:
sample_log.rename({'Salinity [psu]':'Discrete Salinity [psu]'},axis='columns',inplace=True)
sample_log.drop(['Station #','Sample ID'],axis='columns',inplace=True)

In [ ]:
sample_log.rename(columns=lambda x: x.strip(),inplace=True)

In [ ]:
oxy.head()

In [ ]:
sample_log = sample_log.merge(oxy[['Station #','Sample ID','Oxygen [mL/L]']], how='left', left_on=['Station-Cast #','Oxygen Bottle #'], right_on=['Station #','Sample ID'])

In [ ]:
sample_log.rename({'Oxygen [mL/L]':'Discrete Oxygen [mL/L]'},axis='columns',inplace=True)
sample_log.drop(['Station #','Sample ID'],axis='columns',inplace=True)

In [ ]:
nutrients.reset_index(inplace=True)

In [ ]:
nutrients.rename({'index':'Sample ID'},axis='columns',inplace=True)

In [ ]:
sample_log = sample_log.merge(nutrients, how='left', left_on=['Nitrate Bottle 1'], right_on=['Sample ID'])

In [ ]:
sample_log.rename(columns=lambda x: x.replace('Avg:', 'Discrete'), inplace=True)
sample_log.drop(['Sample ID'],axis='columns',inplace=True)

In [ ]:
sample_log.columns.values

In [ ]:
# Now add the chlorophyll data
chl.columns.values

In [ ]:
chl_df = chl[['Station-Cast #','Brown Bottle #','Chl (ug/l)','Phaeo (ug/l)']]
chl_df.rename(columns=lambda x: 'Discrete ' + x, inplace=True)
#chl_df.rename({'Discrete quality_flag':'Discrete Chl quality flag'},axis='columns',inplace=True)

In [ ]:
sample_log = sample_log.merge(chl_df, how='left', left_on=['Station-Cast #','Chlorophyll Brown Bottle #'], right_on=['Discrete Station-Cast #','Discrete Brown Bottle #'])
sample_log.drop(['Discrete Station-Cast #','Discrete Brown Bottle #'],axis='columns',inplace=True)

In [ ]:
# Now load the CTD summary data
CTD = pd.read_csv(basepath+array+cruise+'Leg_1/'+ctd_dir+'CTD_Summary.csv')

In [ ]:
column_list = []
for name in list(sample_log.columns.values):
    if 'Discrete' in name:
        column_list.append(name)
column_list.append('Station-Cast #')
column_list.append('Niskin #')

In [ ]:
discrete_data = sample_log[column_list]

In [ ]:
discrete_data

In [ ]:
CTD = CTD.merge(discrete_data, how='left', left_on=['Cast #','Niskin #'], right_on=['Station-Cast #','Niskin #'])

In [ ]:
CTD.head()

In [ ]:
CTD.drop(['Unnamed: 0'],axis='columns',inplace=True)

In [ ]:
CTD.fillna(-9999999, inplace=True)
CTD['Date Time'] = CTD['Date Time'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%dT%H:%M:%SZ'))

In [ ]:
CTD.head()

In [ ]:
summary_sheet_path

In [ ]:
summary_name = summary_sheet_path.replace('sampling_logs.xlsx','Sample_Summary.csv')
summary_name

In [ ]:
CTD.to_csv(summary_name)

In [ ]:
nutrient_bottles = sample_log['Sample Log: Nitrate Bottle 1'].str.split(',').apply(pd.Series, 1).stack()
nutrient_bottles.index = nutrient_bottles.index.droplevel(-1)
nutrient_bottles.name = 'Nitrate Bottle #'

In [ ]:
nutrient_bottles

In [ ]:
# Add the nutrient bottle number back into the sample log, and remove the excess '.'
sample_log = sample_log.join(nutrient_bottles)
sample_log['Nitrate Bottle #'] = sample_log['Nitrate Bottle #'].apply(lambda x: x.replace('.','') if type(x) == str else x)
sample_log['Nitrate Bottle #'] = sample_log['Nitrate Bottle #'].apply(lambda x: x.replace(' ','') if type(x) == str else x)

In [ ]:
# Now I can add the nutrient bottle data to the sample log before loading
sample_log = sample_log.merge(nutrients, how='left', left_on='Nitrate Bottle #', right_on='Sample ID')

In [ ]:
sample_log.columns.values

In [ ]:
sample_columns = ['Sample Log: Cast #','Sample Log: Niskin #','SAL: Salinity','OXY: Oxy','Avg: Nitrate+Nitrite [µmol/L]', 'Avg: Ammonium [µmol/L]',
    'Avg: Phosphate [µmol/L]', 'Avg: Silicate [µmol/L]', 'Avg: Nitrite [µmol/L]', 'Avg: Nitrate [µmol/L]']

In [ ]:
result = df.merge(sample_log[sample_columns], how='left',
                  left_on=['Cast #','Niskin #'], right_on=['Sample Log: Cast #','Sample Log: Niskin #'])

In [ ]:
result

In [ ]:
result.columns.values

In [ ]:
sal_unit = '[' + list(set(sal_df['SAL: Unit']) )[0] + ']'
oxy_unit = '[' + list(set(oxy_df['OXY: Unit']) )[0] + ']'
sal_unit, oxy_unit

In [ ]:
result.drop(['Sample Log: Cast #','Sample Log: Niskin #'], axis=1, inplace=True)

In [ ]:
result.rename(columns=lambda x: x.replace('SAL:','Bottle') + ' ' + sal_unit if 'SAL:' in x else x, inplace=True)
result.rename(columns=lambda x: x.replace('OXY:','Bottle') + ' ' + oxy_unit if 'OXY:' in x else x, inplace=True)
result.rename(columns=lambda x: x.replace('Avg:','Bottle') if 'Avg:' in x else x, inplace=True)

In [ ]:
result

In [ ]:
# Replace all of the nans with -999999 and save to a csv
result.fillna(str(-9999999),inplace=True)

In [ ]:
result

In [ ]:
result.to_csv(salts_and_o2_path+'Irminger-3_Summary.csv')

In [ ]:
os.getcwd()